In [1]:
import json
import pandas as pd
import numpy as np
from config import username
from config import passwd
from sqlalchemy import create_engine, inspect

In [2]:
# File path
filepath = 'data/yelp_academic_dataset_business.json'

In [3]:
# Read JSON file and extract raw data
raw_data_df = pd.read_json(filepath, lines=True)
raw_data_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


In [4]:
# Make a copy and drop unneeded columns
copy_df = raw_data_df[['name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'categories']].copy().dropna()
copy_df.head()

,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories
0,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,"Gastropubs, Food, Beer Gardens, Restaurants, B..."
1,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,"Salad, Soup, Sandwiches, Delis, Restaurants, C..."
2,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,"Antiques, Fashion, Used, Vintage & Consignment..."
3,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,"Beauty & Spas, Hair Salons"
4,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,"Gyms, Active Life, Interval Training Gyms, Fit..."


In [5]:
# Filter for 'Restaurants' in 'Porltand'
criteria_df = copy_df[(copy_df['categories'].str.contains('Restaurants')) & (copy_df['city'] == 'Portland')]
criteria_df.head()

,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories
1,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,"Salad, Soup, Sandwiches, Delis, Restaurants, C..."
36,Cleary's Restaurant & Spirits,12429 NE Glisan St,Portland,OR,97230,45.526473,-122.535323,3.5,19,"Nightlife, Sandwiches, Seafood, Restaurants"
50,Cafe Yumm!,301 SW Morrison St,Portland,OR,97204,45.518450,-122.675723,3.5,158,"Vegetarian, Vegan, Sandwiches, Soup, American ..."
93,Whole Time Chicken,,Portland,OR,97215,45.516241,-122.597527,5.0,7,"Food Stands, Restaurants, Sandwiches"
94,B Street Coffee House,2190 W Burnside St,Portland,OR,97210,45.523074,-122.696206,4.5,131,"Restaurants, Food, Sandwiches, Breakfast & Bru..."


In [6]:
# Reset the index
clean_df = criteria_df.reset_index(drop=True)
clean_df

,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories
0,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,"Salad, Soup, Sandwiches, Delis, Restaurants, C..."
1,Cleary's Restaurant & Spirits,12429 NE Glisan St,Portland,OR,97230,45.526473,-122.535323,3.5,19,"Nightlife, Sandwiches, Seafood, Restaurants"
2,Cafe Yumm!,301 SW Morrison St,Portland,OR,97204,45.518450,-122.675723,3.5,158,"Vegetarian, Vegan, Sandwiches, Soup, American ..."
3,Whole Time Chicken,,Portland,OR,97215,45.516241,-122.597527,5.0,7,"Food Stands, Restaurants, Sandwiches"
4,B Street Coffee House,2190 W Burnside St,Portland,OR,97210,45.523074,-122.696206,4.5,131,"Restaurants, Food, Sandwiches, Breakfast & Bru..."
...,...,...,...,...,...,...,...,...,...,...
5725,McMenamins Barley Mill Pub,1629 S.E. Hawthorne Blvd.,Portland,OR,97214,45.512422,-122.648782,3.5,108,"Pubs, Breweries, Food, American (Traditional),..."
5726,Seaplane,2266 NW Lovejoy St,Portland,OR,97210,45.529592,-122.697685,3.5,13,"Fashion, Men's Clothing, Restaurants, Shopping..."
5727,The Italian Joint,3145 SE Hawthorne Blvd,Portland,OR,97203,45.512196,-122.632865,3.5,20,"Italian, Restaurants"
5728,Chart House,5700 SW Terwilliger,Portland,OR,97239,45.483154,-122.682748,3.5,457,"Event Planning & Services, Steakhouses, Seafoo..."


In [7]:
# Connect to local database
rds_connection_string = f'{username}:{passwd}@localhost:5432/restaurant_db'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [8]:
# Check engine table names
insp = inspect(engine)
insp.get_table_names()

['yelp_data']